# **Data Science - Principles and applications - Final project - Due date 04/12/2025**
__________
### Analyzing TCL validation data to extract typical day profiles and to highlight days that show abnormal usage due to disruptions, events, or changes in mobility habits

R. ARNAUD, M. DELPLANQUE, E. GASTARD (ENTPE)

Lecturers: A. FURNO, T. LEMOALLE (ENTPE / UGE)

**Dataset : Public Transport Validation Data (SYTRAL) - Time period : from 1 November 2019 to 30 March 2020**
__________

## **1. Preliminary works**

Preliminary works done on the datasets to make the study possible.

### 1.1. Import of the necessary libraries

In [ ]:
# Import necessary core libraries
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import sys,os

# Import plotting functions
import plot_functions as pf
#from filter_definitions import *



# Load files from src folder
FOLDER_PATH = '../'
src_path = Path(FOLDER_PATH) / 'src'
print(f"Adding {src_path} to sys.path")
sys.path.append(str(src_path))

# Extract dataframes from dataframe_extraction.py in src folder

from dataframe_extraction import df_bus , df_tramway

ModuleNotFoundError: No module named 'plot_functions'

### 1.2. Paths and data opening

In [10]:
# === Paths & Data ===
FOLDER_PATH = '..'
DATA_PATH= os.path.join(FOLDER_PATH, 'data') 
display(os.getcwd())

'c:\\Users\\romai\\OneDrive\\Documents\\_ENTPE\\Formation\\Data science\\Data Science - Principles and applications (A. FURNO)\\Project\\dspap_project_repo\\notebooks'

Il faut des ofnctions separees et commentee pour les fonctions : pre processing, feature enginering, preliminary data analysis & statistics, machine learning task

In [ ]:
# === Libraries ===
# Core numerical & scientific libraries
import numpy as np                   # Work with arrays
import matplotlib.pyplot as plt      # Visualization and plotting
import os

# === Paths & Data ===
#Root_path  ='..'
Root_path='dspap_project-repo'
Folder_path='data'
data_path= os.path.join(Root_path, Folder_path)  # Build full image path
display(os.getcwd())

# Load files from src folder
src_path = Path(FOLDER_PATH) / 'src'
print(f"Adding {src_path} to sys.path")
sys.path.append(str(src_path))

'c:\\Users\\romai\\OneDrive\\Documents\\_ENTPE\\Formation\\Data science\\Data Science - Principles and applications (A. FURNO)\\Project'

### 1.3. Import the pands filters

In [ ]:
import sys
sys.path.append("../src")

import filters

# Covid
filter_no_covid_tram = filters.filter_no_covid(df_tramway)
filter_no_covid_bus  = filters.filter_no_covid(df_bus)

# Christmas holidays
filter_christmas_holiday_tram = filters.filter_christmas_holiday(df_tramway)
filter_christmas_holiday_bus  = filters.filter_christmas_holiday(df_bus)

# February holidays
filter_february_holiday_tram = filters.filter_february_holiday(df_tramway)
filter_february_holiday_bus  = filters.filter_february_holiday(df_bus)

# Fête des Lumières
filter_fete_lumieres_tram = filters.filter_fete_lumieres(df_tramway)
filter_fete_lumieres_bus  = filters.filter_fete_lumieres(df_bus)




ModuleNotFoundError: No module named 'src'

In [18]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
FOLDER_PATH = '..'
base = Path(FOLDER_PATH) / "data" / "PT" / "pt_data" # Base path for the data files

agg = "15min"  
modes = ["subway", "tramway", "bus"]


dfs = {}  # mode -> DataFrame

for mode in ['subway','tramway','bus']:
    csv_path = f"{base}/{mode}_indiv_{agg}/{mode}_indiv_{agg}.csv"
    df = pd.read_csv(csv_path,index_col = 0, low_memory=False)
    if 'VAL_DATE' in df.columns:
        df['VAL_DATE'] = pd.to_datetime(df['VAL_DATE']) 
    else: 
        df.index = pd.to_datetime(df.index)
    dfs[mode] = df
    



   


In [ ]:
ref_subway = pd.read_csv(f"{base}/ref_subway.csv",index_col = 0).rename(columns = {'MEAN_X' : 'lon','MEAN_Y':'lat'})
ref_tram_bus = pd.read_csv(f"{base}/ref_tram_bus.csv",index_col = 0).rename(columns = {'IDT_PNT' : 'VAL_ARRET_CODE','COO_X_WGS84':'lon','COO_Y_WGS84':'lat'})

ref_tram_bus = ref_tram_bus[['lon','lat','NOM_PNT','VAL_ARRET_CODE']]
ref_subway = ref_subway[['lon','lat','COD_TRG','LIB_STA_SIFO']]

df_bus = dfs["bus"].merge(ref_tram_bus, how = 'inner', on = 'VAL_ARRET_CODE')  
df_tramway = dfs["tramway"].merge(ref_tram_bus, how = 'inner', on = 'VAL_ARRET_CODE')

df_bus["date"]=pd.to_datetime(df_bus["VAL_DATE"])
df_tramway["date"]=pd.to_datetime(df_tramway["VAL_DATE"])

df_bus["date_only"] = df_bus["VAL_DATE"].dt.date
df_tramway["date_only"] = df_tramway["VAL_DATE"].dt.date


In [19]:
for mode in ['subway','tramway','bus']:
    display(dfs[mode].head()) 

,AMP,BEL,BRO,CHA,COR,CPA,CRO,CUI,CUS,DEB,...,PER,GUI,JAU,REP,SAN,SAX,GER,VMY,SOI,JEA
VAL_DATE,,,,,,,,,,,,,,,,,,,,,
2019-11-01 00:00:00,20.0,164.0,10.0,59.0,88.0,5.0,20.0,1.0,5.0,21.0,...,89.0,13.0,16.0,12.0,49.0,68.0,8.0,23.0,3.0,59.0
2019-11-01 00:15:00,6.0,82.0,11.0,30.0,43.0,3.0,8.0,0.0,2.0,3.0,...,19.0,7.0,7.0,3.0,11.0,33.0,2.0,16.0,0.0,38.0
2019-11-01 00:30:00,0.0,4.0,0.0,0.0,2.0,2.0,6.0,0.0,1.0,0.0,...,0.0,0.0,0.0,3.0,5.0,2.0,0.0,3.0,1.0,0.0
2019-11-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-11-01 04:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0


,LIG_NUMERO_SAE,VAL_ARRET_CODE,CRS_SENS_TRAJET,COD_LIG_CLI,VAL_DATE,Flow
0,520,32102,0,T1,2019-11-02 15:00:00,66
1,520,32102,0,T1,2019-11-02 17:30:00,53
2,520,32102,0,T1,2019-11-02 17:45:00,72
3,520,32102,0,T1,2019-11-02 18:30:00,26
4,520,32102,0,T1,2019-11-03 04:45:00,1


,LIG_NUMERO_SAE,VAL_ARRET_CODE,CRS_SENS_TRAJET,COD_LIG_CLI,VAL_DATE,Flow
0,2,202,1,2,2019-11-01 12:15:00,1
1,2,202,1,2,2019-11-02 10:30:00,1
2,2,202,1,2,2019-11-02 12:00:00,1
3,2,202,1,2,2019-11-04 08:00:00,1
4,2,202,1,2,2019-11-04 08:15:00,1
